<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLNN_func_approx_050422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# data

In [2]:
data=[
[1127,	867,	850,	783,	1196,	1067,	962,	1543,	1430,	1544,	1440,	931,	770,	734,	655,	757,	2033,	1586,	1767,	1393,	1594,	1745,],
[1.313,	1.373,	1.389,	1.408,	1.293,	1.326,	1.355,	1.237,	1.25,	1.263,	1.253,	1.364,	1.406,	1.44,	1.46,	1.427,	1.207,	1.239,	1.22,	1.246,	1.25,	1.222,]
]
data=np.array(data).T
data=pd.DataFrame(data, columns=['sol','den']).sort_values(by=['sol'])
data.head()

,sol,den
14,655.0,1.460
13,734.0,1.440
15,757.0,1.427
12,770.0,1.406
3,783.0,1.408


In [3]:
pip install MLNN

  Created wheel for MLNN: filename=MLNN-1.1-py3-none-any.whl size=5006 sha256=8d3f47c19b02f5623907c113e5ab70437530ac7a1188eb7e13ea371650778283
  Stored in directory: /root/.cache/pip/wheels/1c/33/14/5e878c08d99131b6dcfc8738c355e73fa081a3fb81bb3b7f3d
Successfully built MLNN


# scaling

In [4]:
from sklearn import preprocessing
from MLNN import *

In [5]:
scaler_X = preprocessing.StandardScaler().fit(data[['sol']])
print(scaler_X.mean_, scaler_X.scale_)
X_sc = scaler_X.transform(data[['sol']]) # matrix
X_sc[0]

[1217.] [395.92572765]


array([-1.41945815])

In [6]:
scaler_y = preprocessing.StandardScaler().fit(data[['den']])
y_sc = scaler_y.transform(data[['den']])[:,0] # vector
y_sc

array([ 1.80801329,  1.55776923,  1.3951106 ,  1.13235434,  1.15737875,
        0.9196469 ,  0.71945165,  0.60684183,  0.49423201,  0.13137813,
       -0.03128051, -0.28152456, -0.86959809, -0.81954928, -0.78201267,
       -0.98220791, -0.65689064, -0.95718351, -0.81954928, -1.16989095,
       -1.19491536, -1.35757399])

# model fitting

In [7]:
layers=[1,2,1]

In [8]:
reg1=Regres(layers=layers)
reg1.set_hidden_activ('mrelu', param=0.01)
reg1.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.01)
data1=go.Scatter(y=reg1.MSE_history[:], name='eta=0.01')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [9]:
reg2=Regres(layers=layers)
reg2.set_hidden_activ('mrelu', param=0.01)
reg2.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.001)
data2=go.Scatter(y=reg2.MSE_history[:], name='eta=0.001')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [12]:
reg_fixed=Regres(layers=layers)
reg_fixed.set_hidden_activ('mrelu', param=0.01)
reg_fixed.set_weights(fixed_weights) # if no fixed_weights exist yet, skip it
reg_fixed.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.0001)
data_fixed=go.Scatter(y=reg_fixed.MSE_history[:], name='fixed')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [23]:
fig1=go.Figure()
fig1.add_trace(data1), print('MSE1=', reg1.MSE_history[-1])
fig1.add_trace(data2), print('MSE2=', reg2.MSE_history[-1])

fig1.add_trace(data_fixed), print('MSE_fixed=', reg_fixed.MSE_history[-1])

fig1.update_yaxes(range=[0, 1])
fig1.show()

MSE1= 0.3123034489339178
MSE2= 0.042441781347514015
MSE_fixed= 0.03600228033876645


# model fixation

In [14]:
reg2.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.97621267 0.10732528] mrelu
Layer 1 Neuron 1 : [-1.2359332   0.01937847] mrelu
Layer 2 Neuron 0 : [-0.45335551  1.22185864 -0.46634052] linear



In [15]:
reg2.weight_history[-1]

[[[0.9762126650907305, 0.10732528286532016],
  [-1.2359331962740436, 0.019378471578856162]],
 [[-0.4533555134193632, 1.2218586415631227, -0.46634052262367076]]]

In [16]:
fixed_weights=[[[-1.4199247975372826, 0.38411881911459445],
  [0.7527867259577954, -0.5721809049933203]],
 [[1.036647602612719, -0.6109860893233047, -0.822373385679829]]]
fixed_MSE=0.03904148189269577

# predictions

In [17]:
chosen_model=reg_fixed
# chosen_model=reg2

In [18]:
predics=scaler_y.inverse_transform([chosen_model.predict(np.array(X_sc))])[0]
errors=(predics-data.den)/data.den*100

In [27]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data.sol, y=data.den, mode='markers', name='data'),secondary_y=False)
fig.add_trace(go.Scatter(x=data.sol, y=predics, mode='lines', name='prediction'),secondary_y=False)
fig.add_trace(go.Scatter(x=data.sol, y=errors, mode='markers', name='errors'),secondary_y=True)

fig.update_xaxes(title_text="R(Sol)")

# Set y-axes titles
fig.update_yaxes(title_text="Density", secondary_y=False)
fig.update_yaxes(title_text="% error", range=[-5, 5], secondary_y=True, showgrid=False, tick0=-5, dtick=2)

fig.show()